In [3]:
! pip install yfinance

In [4]:
! pip install pandas_datareader

# Importing necessary libraries

In [3]:
from bs4 import BeautifulSoup as BS
import requests
import yfinance as yf
import datetime
import pandas as pd
import pandas_datareader.data as web

import urllib.request as urllib2 
from urllib.request import URLError

from tqdm import tqdm

import os 
import pickle
import numpy as np

import copy
import math


# Getting the list of tickers

In [4]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BS(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2021,4,28)
end = datetime.datetime(2021,5,28)


# Getting market cap data for each ticker

In [5]:
dates_keys = ['2021-04-28', '2021-04-29', '2021-04-30',  '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06']
dates_keys = dates_keys + ['2021-05-07', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19']
dates_keys = dates_keys + ['2021-05-20', '2021-05-21', '2021-05-24', '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28']


# Try to load data 

In [6]:
if 'market_cap.pkl' in os.listdir('.'):
  with open('market_cap.pkl', 'rb') as file:
    market_cap_dict = pickle.load(file)

# If there is no data to load -> download it yourself

In [7]:
if 'market_cap.pkl' not in os.listdir('.'):

  market_cap_dict = {}

  for ticker in tqdm(tickers):
    soup = BS(urllib2.urlopen('https://ycharts.com/companies/' + ticker + '/market_cap'), features="html.parser")
    leftCol = soup.find('div', {'class':'dataColLeft'})
    dates = leftCol.find_all('td', {'class':'col1'})
    market_caps = leftCol.find_all('td', {'class':'col2'})

    dates = list(map(lambda x: x.string, dates))
    market_caps = list(map(lambda x: x.string.strip(), market_caps))

    zipped = list(zip(dates, market_caps))

    # preprocess the data
    dates = []
    for zipp in zipped:
      month_day, year = zipp[0].split(', ')
      month, day = month_day.split(' ')
      dates.append((year + ('-05-' if month == 'May' else '-04-') + ('0' if int(day) < 10 else '') + day))

    prices = []
    for zipp in zipped:
      value = int(zipp[1].replace('.', '').replace('T', '000000000').replace('B', '000000'))
      prices.append(value)

    marketCapKnown = dict(list(zip(dates, prices)))

    marketCap = dict([(dk, 0) for dk in dates_keys])
    marketCap.update(marketCapKnown)

    market_cap_dict[ticker] = marketCap


# Save data to reuse it later

In [8]:
if 'market_cap.pkl' not in os.listdir('.'):
   with open('market_cap.pkl', 'wb') as file:
     pickle.dump(market_cap_dict, file)

# Download yfinance data

In [10]:
# if 'yf_downloas_data.pkl' in os.listdir('.'):
#    with open('yf_downloas_data.pkl', 'rb') as file:
#      data = pickle.load(file)

In [11]:
data = yf.download(tickers, start=start, end=end)


[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [12]:
# if 'yf_downloas_data.pkl' not in os.listdir('.'):
#    with open('yf_downloas_data.pkl', 'wb') as file:
#      pickle.dump(data, file)

In [13]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Get Adj Close data

In [14]:
adj_close_data = data['Adj Close']

# Calculating change in percentages

In [15]:
ptc_changes = adj_close_data.pct_change()
ptc_changes = ptc_changes.dropna(how='all')
ptc_changes = ptc_changes.dropna(axis=1,how='all')
ptc_changes.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,CPB,CPRT,CRL,CRM,CSCO,CSX,CTAS,CTLT,CTSH,CTVA,CTXS,CVS,CVX,CZR,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,ENPH,EOG,EQIX,EQR,ES,ESS,ETN,ETR,ETSY,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLT,FMC,FOX,FOXA,FRC,FRT,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GNRC,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,J,JBHT,JCI,JKHY,JNJ,JNPR,JPM,K,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KR,KSU,L,LB,LDOS,LEG,LEN,LH,LHX,LIN,LKQ,LLY,LMT,LNC,LNT,LOW,LRCX,LUMN,LUV,LVS,LW,LYB,LYV,MA,MAA,MAR,MAS,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOS,MPC,MPWR,MRK,MRO,MS,MSCI,MSFT,MSI,MTB,MTD,MU,MXIM,NCLH,NDAQ,NEE,NEM,NFLX,NI,NKE,NLOK,NLSN,NOC,NOV,NOW,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,NXPI,O,ODFL,OKE,OMC,ORCL,ORLY,OTIS,OXY,PAYC,PAYX,PBCT,PCAR,PEAK,PEG,PENN,PEP,PFE,PFG,PG,PGR,PH,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,POOL,PPG,PPL,PRGO,PRU,PSA,PSX,PTC,PVH,PWR,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RJF,RL,RMD,ROK,ROL,ROP,ROST,RSG,RTX,SBAC,SBUX,SCHW,SEE,SHW,SIVB,SJM,SLB,SNA,SNPS,SO,SPG,SPGI,SRE,STE,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYY,T,TAP,TDG,TDY,TEL,TER,TFC,TFX,TGT,TJX,TMO,TMUS,TPR,TRMB,TROW,TRV,TSCO,TSLA,TSN,TT,TTWO,TWTR,TXN,TXT,TYL,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,V,VFC,VIAC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-29,-0.004822,-0.018433,0.015433,-0.000749,-0.009292,0.016090,-0.080388,-0.003623,0.007820,0.001183,0.012495,0.007290,0.009538,-0.009859,0.005858,0.015681,0.002912,0.013079,0.015142,0.012291,0.004926,0.009359,-0.010508,-0.014706,-0.001165,0.020104,0.005275,-0.006897,-0.002211,0.017109,-0.001309,0.014955,-0.008449,0.016445,-0.006112,0.003704,0.005456,-0.009590,0.020651,0.003734,0.032136,0.009273,0.009358,0.016588,-0.016199,0.006554,0.018744,0.009205,0.000836,0.015440,0.015704,0.000648,0.013701,0.010247,0.002039,0.026980,-0.018853,0.009386,-0.023650,0.015599,-0.008483,-0.012804,0.018300,-0.001465,0.014321,0.013525,0.022222,-0.047554,0.002795,-0.003165,-0.021799,0.017482,0.012755,0.006673,0.009654,-0.007642,-0.020792,0.033159,0.006134,0.030497,0.014958,-0.021223,0.001426,0.002582,0.010244,0.004935,0.001412,0.018406,0.018720,0.040585,0.026428,0.012544,0.044399,0.013330,0.004701,0.028081,0.042514,0.013481,0.005386,0.005482,0.014274,0.018490,0.023809,0.033735,0.005304,0.001302,0.001697,0.012852,0.009460,0.019387,-0.002778,-0.010638,0.006848,-0.003054,0.016976,0.006183,0.014717,0.007522,-0.075735,0.012865,0.015967,-0.030507,0.017774,-0.001292,0.011291,-0.008978,0.025556,0.013155,-0.001671,0.016632,-0.007005,0.010579,0.012270,0.011577,0.083193,0.012706,0.012189,0.021810,0.011846,0.029781,0.012591,0.016334,0.021819,0.009942,0.010237,0.001665,0.048620,-0.009559,0.016321,-0.100289,0

In [16]:
number_of_companies = len(ptc_changes.columns)
changes_of_prices = ptc_changes.sum(axis=1)

index_change_ptc = changes_of_prices/number_of_companies+1

In [17]:
index_change_ptc_df=pd.DataFrame(index_change_ptc)

# Projecting and comparing the projection to the real values

In [18]:
SP500 = web.DataReader(['sp500'], 'fred', start, end)

In [19]:
real_SP500 = SP500.dropna(axis=0,how='all')
starting_sp500_value = real_SP500.head().values[0][0]

In [20]:
sp_curr = starting_sp500_value  # s&p index value on the starting date
sp_list=[sp_curr]
for ind in index_change_ptc:
  sp_curr=sp_curr*ind
  sp_list.append(sp_curr)

In [21]:
sp_projection_df = pd.DataFrame(sp_list, index=real_SP500.index[:-1])

In [22]:
comparison_ew=pd.concat([real_SP500,sp_projection_df],axis=1)
comparison_ew


,sp500,0
DATE,,
2021-04-28,4183.18,4183.180000
2021-04-29,4211.47,4212.774420
2021-04-30,4181.17,4179.321072
2021-05-03,4192.66,4204.586697
2021-05-04,4164.66,4197.610130
2021-05-05,4167.59,4209.624691
2021-05-06,4201.62,4230.443724
2021-05-07,4232.60,4275.623364
2021-05-10,4188.43,4257.427008


## Equal weighed influence is not the best approach.

#### Because of that, we are going to calculate the weight for each ticker as the ratio of it's number of shares and the total number of shares of all tickers.

\begin{align}
W_{ticker} & = \frac{number\_of\_shares_{ticker}}{total\_number\_of\_shares}
\end{align}



# Involving market capitalization

In [23]:
market_cap_df = pd.DataFrame.from_dict(market_cap_dict)

# sort rows by date
market_cap_df = market_cap_df.sort_index()
# sort columns by name (i.e date)
market_cap_df = market_cap_df.reindex(sorted(market_cap_df.columns), axis=1)




In [24]:
# removing non-interesting dates 
# market_cap_df = market_cap_df.iloc[3:-1]

# Dividing the market capitalization with stock price to get the number of shares on the given date


\begin{align}
number\_of\_shares_{ticker} & = \frac{market\_cap_{ticker}}{share\_price_{ticker}}
\end{align}



In [25]:
number_of_shares_per_ticker_df = market_cap_df.iloc[1:-1].div(adj_close_data)

# Calculating the ratio of the number-of-shares per ticker and total-number-of-shares

In [26]:
number_of_shares_per_ticker_list = number_of_shares_per_ticker_df.values.tolist()
number_of_shares_per_ticker_list = list(map(lambda list_: [0 if math.isnan(i) else i for i in list_], number_of_shares_per_ticker_list))
number_of_shares_per_ticker_sums = list(map(sum, number_of_shares_per_ticker_list))

number_of_shares_ptc = copy.copy(number_of_shares_per_ticker_df)

for i in range(len(number_of_shares_per_ticker_df)):
  number_of_shares_ptc.iloc[i] = number_of_shares_per_ticker_df.iloc[i] / number_of_shares_per_ticker_sums[i]


In [27]:
number_of_shares_ptc

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,CPB,CPRT,CRL,CRM,CSCO,CSX,CTAS,CTLT,CTSH,CTVA,CTXS,CVS,CVX,CZR,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,ENPH,EOG,EQIX,EQR,ES,ESS,ETN,ETR,ETSY,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLT,FMC,FOX,FOXA,FRC,FRT,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GNRC,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,J,JBHT,JCI,JKHY,JNJ,JNPR,JPM,K,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KR,KSU,L,LB,LDOS,LEG,LEN,LH,LHX,LIN,LKQ,LLY,LMT,LNC,LNT,LOW,LRCX,LUMN,LUV,LVS,LW,LYB,LYV,MA,MAA,MAR,MAS,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOS,MPC,MPWR,MRK,MRO,MS,MSCI,MSFT,MSI,MTB,MTD,MU,MXIM,NCLH,NDAQ,NEE,NEM,NFLX,NI,NKE,NLOK,NLSN,NOC,NOV,NOW,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,NXPI,O,ODFL,OKE,OMC,ORCL,ORLY,OTIS,OXY,PAYC,PAYX,PBCT,PCAR,PEAK,PEG,PENN,PEP,PFE,PFG,PG,PGR,PH,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,POOL,PPG,PPL,PRGO,PRU,PSA,PSX,PTC,PVH,PWR,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RJF,RL,RMD,ROK,ROL,ROP,ROST,RSG,RTX,SBAC,SBUX,SCHW,SEE,SHW,SIVB,SJM,SLB,SNA,SNPS,SO,SPG,SPGI,SRE,STE,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYY,T,TAP,TDG,TDY,TEL,TER,TFC,TFX,TGT,TJX,TMO,TMUS,TPR,TRMB,TROW,TRV,TSCO,TSLA,TSN,TT,TTWO,TWTR,TXN,TXT,TYL,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,V,VFC,VIAC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2021-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Do the dot product of the price change and calculated weights


#### First, make sure that the lists of tickers match completely

In [28]:
columns_for_removal = []
for share_ticker in number_of_shares_ptc.columns:
  if share_ticker not in ptc_changes.columns:
    columns_for_removal.append(share_ticker)

number_of_shares_ptc = number_of_shares_ptc.drop(columns_for_removal, axis=1)


#### Finally, calculate the dot product

In [29]:
number_of_shares_ptc_list = number_of_shares_ptc.values.tolist()
ptc_change_list = ptc_changes.values.tolist()

lists = list(zip(number_of_shares_ptc_list, ptc_change_list))
weighted_change = list(map(lambda x: 1+sum(i[0] * i[1] for i in zip(x[0], x[1])), lists))
weighted_change


[nan,
 0.990074243833688,
 1.0067497310800289,
 0.9875679720178926,
 1.0040449654826296,
 1.009505339196518,
 1.0087024600392707,
 0.9878278099403269,
 0.9901595780024103,
 0.9787340494065361,
 1.0133143735446621,
 1.018979980816219,
 0.9980522036390922,
 0.989500964614387,
 0.9954132415915954,
 1.0104533620343064,
 0.9970833399441141,
 1.0097625182484409,
 0.9955337084843084,
 1.003852858301966,
 1.0002041894977942]

# Comparing outputs

In [38]:
sp_curr = starting_sp500_value  # s&p index value on the starting date
sp_list=[sp_curr]
for ind in weighted_change[1:]:
  sp_curr=sp_curr*ind
  sp_list.append(sp_curr)


In [40]:
weighted_sp_projection_df = pd.DataFrame(sp_list, index=real_SP500.index[:-2])

In [41]:
comparison=pd.concat([ weighted_sp_projection_df, comparison_ew],axis=1)
comparison.set_axis(['Weighted Projection', 'Real S&P500', 'Equal Weighted Projection'], axis=1, inplace=True)
comparison

,Weighted Projection,Real S&P500,Equal Weighted Projection
DATE,,,
2021-04-28,4183.180000,4183.18,4183.180000
2021-04-29,4141.658775,4211.47,4212.774420
2021-04-30,4169.613858,4181.17,4179.321072
2021-05-03,4117.777102,4192.66,4204.586697
2021-05-04,4134.433368,4164.66,4197.610130
2021-05-05,4173.732560,4167.59,4209.624691
2021-05-06,4210.054301,4201.62,4230.443724
2021-05-07,4158.808720,4232.60,4275.623364
2021-05-10,4117.884287,4188.43,4257.427008


## Calculating the squared error

In [44]:
sum_sq_w = 0
sum_sq_e = 0

for triple in comparison.values.tolist()[:-2]:
  sum_sq_w += (triple[0] - triple[1])**2
  sum_sq_e += (triple[2] - triple[1])**2

length = len(comparison.values.tolist())
sum_sq_w /= length
sum_sq_e /= length  

In [46]:

np.sqrt(sum_sq_w), np.sqrt(sum_sq_e)
print('Equal-weigth-projection squared error: ', np.sqrt(sum_sq_e))
print('Weighted-projection squared error: ', np.sqrt(sum_sq_w))

Equal-weigth-projection squared error:  45.15503109570736
Weighted-projection squared error:  47.15978428340358


# Linear Regression

In [47]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [48]:
tmp = adj_close_data[1:]

features = tmp.values
features = np.delete(features, 60, 1)
features = np.delete(features, 69, 1)

featuresList = np.ndarray.tolist(features)

for row in featuresList:
  for el in row:
    if not math.isfinite(el):
      print(featuresList.index(row), row.index(el))


In [49]:
targets = real_SP500.values[:-1]

In [50]:
degree = 1
poly = PolynomialFeatures(degree)

features_poly = poly.fit_transform(features)

train_features, test_features, train_targets, test_targets = train_test_split(
    features_poly, targets, test_size=0.33 
)

In [51]:
# Train
linear_regressor = LinearRegression()
linear_regressor.fit(train_features, train_targets)

test_preds_lr = linear_regressor.predict(test_features)
rmse_lr = mean_squared_error(test_targets, test_preds_lr, squared=False)

print("LR RMSE -- ", rmse_lr)

for i in range(8):
    print("PRED -- ", test_preds_lr[i])
    print("REAL -- ", train_targets[i])
    print("\n\n")


LR RMSE --  21.267040800361286
PRED --  [4150.42286235]
REAL --  [4192.66]



PRED --  [4226.72576755]
REAL --  [4063.04]



PRED --  [4196.5977915]
REAL --  [4200.88]



PRED --  [4128.89829393]
REAL --  [4164.66]



PRED --  [4092.34308027]
REAL --  [4155.86]



PRED --  [4184.07044887]
REAL --  [4188.43]



PRED --  [4181.71372914]
REAL --  [4181.17]



PRED --  [4170.95729528]
REAL --  [4127.83]



